In [198]:
import pandas as pd
import plotly.express as px


data = pd.read_excel('../data/get_around_delay_analysis.xlsx')
for index, i in enumerate(data.columns):
    print(f"{data.dtypes.iloc[index]} --> {i} ")
print()
display(data[data['delay_at_checkout_in_minutes'].notna()].head())
display(data.shape)

int64 --> rental_id 
int64 --> car_id 
object --> checkin_type 
object --> state 
float64 --> delay_at_checkout_in_minutes 
float64 --> previous_ended_rental_id 
float64 --> time_delta_with_previous_rental_in_minutes 



,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
5,511626,398802,mobile,ended,-203.0,NaN,NaN
6,511639,370585,connect,ended,-15.0,563782.0,570.0
7,512303,371242,mobile,ended,-44.0,NaN,NaN


(21310, 7)

In [199]:
display(data.isna().sum())
display(data["state"].value_counts())


rental_id                                         0
car_id                                            0
checkin_type                                      0
state                                             0
delay_at_checkout_in_minutes                   4964
previous_ended_rental_id                      19469
time_delta_with_previous_rental_in_minutes    19469
dtype: int64

state
ended       18045
canceled     3265
Name: count, dtype: int64

In [200]:
display(data.describe().round(2))

,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.00,21310.00,16346.00,1841.00,1841.00
mean,549712.88,350030.60,59.70,550127.41,279.29
std,13863.45,58206.25,1002.56,13184.02,254.59
min,504806.00,159250.00,-22433.00,505628.00,0.00
25%,540613.25,317639.00,-36.00,540896.00,60.00
50%,550350.00,368717.00,9.00,550567.00,180.00
75%,560468.50,394928.00,67.00,560823.00,540.00
max,576401.00,417675.00,71084.00,575053.00,720.00


In [201]:
print(f"Nombre de voiture : {data.value_counts(subset='car_id').sum()}")
print("Distribution des ended et canceled :")
plot = px.bar(data.value_counts(subset='state').reset_index(),x='state',y='count',title="Distribution ended / canceled",width=400)
plot.show()

Nombre de voiture : 21310
Distribution des ended et canceled :


In [202]:
plot = px.bar(data.value_counts(subset='checkin_type').reset_index(),x='checkin_type',y='count',title="Distribution mobile / connect",width=400)
plot.show()

In [203]:
print("Taux de canceled en fonction du device :")
mobile = data[data['checkin_type'] == 'mobile']
connect = data[data['checkin_type'] == 'connect']

print(f"Taux de canceled quand checkin fait sur appli mobile en présence des deux parties : {(mobile[mobile['state'] == "canceled"]['state'].count() / mobile['rental_id'].count()).round(2)} %")
print(f"Taux de canceled quand checkin fait en connect (sans propriétaire présent ) : {(connect[connect['state'] == "canceled"]['state'].count() / connect['rental_id'].count()).round(2)} %")

Taux de canceled en fonction du device :
Taux de canceled quand checkin fait sur appli mobile en présence des deux parties : 0.15 %
Taux de canceled quand checkin fait en connect (sans propriétaire présent ) : 0.19 %


In [204]:
print("On regarde les données quand le delta delai checkout et inférieur 0 (en avance) :")
checkout_inf_zero = data[data['delay_at_checkout_in_minutes'] <= 0]
print(f"Nombre de reservation inférieur à zéro : {checkout_inf_zero.count()['rental_id']}")
cancel_inf_zero = checkout_inf_zero[checkout_inf_zero['state'] == 'canceled']
print(f"Nombre d'annulation quand en avance : {cancel_inf_zero.count()['rental_id']}")


On regarde les données quand le delta delai checkout et inférieur 0 (en avance) :
Nombre de reservation inférieur à zéro : 6942
Nombre d'annulation quand en avance : 1


On peut donc considérer (et c'est logique) qu'il ne faut regarder que les données ou le delai est supérieur à zéro

In [212]:
print("On regarde les données quand le delta delai checkout et supérieur 0 (en avance) :")
checkout_supp_zero = data[data['delay_at_checkout_in_minutes'] >= 0]
print(checkout_supp_zero.head())
print(f"Nombre de reservation inférieur à zéro : {checkout_supp_zero.count()['rental_id']}")
cancel_supp_zero = checkout_supp_zero[checkout_supp_zero['state'] == 'canceled']
print(f"Nombre d'annulation quand en retard : {cancel_supp_zero.count()['rental_id']}") 


On regarde les données quand le delta delai checkout et supérieur 0 (en avance) :
    rental_id  car_id checkin_type  state  delay_at_checkout_in_minutes  \
2      508131  359049      connect  ended                          70.0   
9      513434  256528      connect  ended                          23.0   
12     514257  402596       mobile  ended                         214.0   
13     515147  257466       mobile  ended                          15.0   
17     518895  398816       mobile  ended                          66.0   

    previous_ended_rental_id  time_delta_with_previous_rental_in_minutes  
2                        NaN                                         NaN  
9                        NaN                                         NaN  
12                       NaN                                         NaN  
13                       NaN                                         NaN  
17                       NaN                                         NaN  
Nombre de reserva

In [206]:
print('Suppression des outliers ...')
data_clean = data.copy()
outliers_col = ['delay_at_checkout_in_minutes', 'previous_ended_rental_id', 'time_delta_with_previous_rental_in_minutes']

for col in outliers_col:
    mean = data_clean[col].mean()
    ecart_type = data_clean[col].std()
    bas, haut = mean - 3*ecart_type, mean + 3*ecart_type
    data_clean = data_clean[(data_clean[col]>= bas) & (data_clean[col] <= haut)]
display(data_clean.shape)
display(data_clean.describe().round(2))
print(data_clean.value_counts(subset='state'))

Suppression des outliers ...


(1500, 7)

,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,1500.00,1500.00,1500.00,1500.00,1500.00
mean,552267.78,351915.62,14.56,550244.28,277.84
std,13013.39,53938.53,302.17,12892.52,254.84
min,505560.00,159533.00,-2676.00,509729.00,0.00
25%,543553.00,324092.00,-47.25,540791.75,60.00
50%,552484.50,368372.50,4.00,550575.00,180.00
75%,562836.75,391618.00,54.00,560843.25,540.00
max,576292.00,415953.00,2943.00,575053.00,720.00


state
ended    1500
Name: count, dtype: int64


In [207]:
box = px.box(data_clean,x='state',y='delay_at_checkout_in_minutes',width=600)
box.show()

In [208]:
print("Analyse des canceled")
data_canceled = data[data["state"] == "canceled"]
print(data_canceled.shape)
display(data_canceled.describe())
display(data_canceled.isna().sum())
plot_canceled = px.box(data_canceled,x='state',y='time_delta_with_previous_rental_in_minutes',width=600)
plot_canceled.show()

Analyse des canceled
(3265, 7)


,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,3265.000000,3265.000000,1.0,229.000000,229.000000
mean,548637.683920,350585.309954,-17468.0,550913.327511,294.890830
std,14907.810897,57254.052866,NaN,11955.397600,250.591601
min,504871.000000,159533.000000,-17468.0,509972.000000,0.000000
25%,539183.000000,317572.000000,-17468.0,543706.000000,60.000000
50%,549700.000000,368593.000000,-17468.0,550970.000000,210.000000
75%,560563.000000,394869.000000,-17468.0,560395.000000,570.000000
max,576195.000000,416935.000000,-17468.0,574540.000000,720.000000


rental_id                                        0
car_id                                           0
checkin_type                                     0
state                                            0
delay_at_checkout_in_minutes                  3264
previous_ended_rental_id                      3036
time_delta_with_previous_rental_in_minutes    3036
dtype: int64

In [209]:
print(data_clean.groupby('checkin_type')['delay_at_checkout_in_minutes'].mean())
print(data_clean.groupby('checkin_type')['delay_at_checkout_in_minutes'].median())

checkin_type
connect   -36.840183
mobile     54.618031
Name: delay_at_checkout_in_minutes, dtype: float64
checkin_type
connect   -12.0
mobile     13.0
Name: delay_at_checkout_in_minutes, dtype: float64


In [210]:
print(data_clean.groupby('state')['checkin_type'].value_counts())
print(data_clean[data_clean["delay_at_checkout_in_minutes"] == 0])

state  checkin_type
ended  mobile          843
       connect         657
Name: count, dtype: int64
       rental_id  car_id checkin_type  state  delay_at_checkout_in_minutes  \
2660      539298  356407       mobile  ended                           0.0   
2901      510586  393908       mobile  ended                           0.0   
4428      543788  377071       mobile  ended                           0.0   
4445      545070  398326       mobile  ended                           0.0   
4871      552075  364038       mobile  ended                           0.0   
5307      559077  355444      connect  ended                           0.0   
6714      565247  406625      connect  ended                           0.0   
8800      529942  385128       mobile  ended                           0.0   
13676     573809  167490       mobile  ended                           0.0   
15063     568500  312603      connect  ended                           0.0   
15275     546943  368432       mobile  end

In [211]:
data_encoded = data.copy()
data_encoded['state'] = data_encoded['state'].apply(lambda x : 1 if x=="ended" else 0)
data_encoded['checkin_type'] = data_encoded['checkin_type'].apply(lambda x : 1 if x=="mobile" else 0)
corr = data_encoded[['state','checkin_type','delay_at_checkout_in_minutes','time_delta_with_previous_rental_in_minutes']].corr('pearson')

corr_df_fig = px.imshow(corr, text_auto=True,
    color_continuous_scale="RdBu_r",
    title="Matrice de corrélation (Pearson)",
    zmin=-1, zmax=1,width=700)
corr_df_fig.show()